In [210]:
import pandas as pd
import nameparser
import json
import requests
import numpy as np

pro_publica_api_key = "PguHImFJakafbsBpZ2ed21A1HIgvXKiM7tg0BqI0 "
headers = {'X-API-Key': pro_publica_api_key}

In [22]:
def get_committees_and_subcommittees(congress, chamber):
    url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees.json'
    response = requests.get(url, headers=headers)

    committees = response.json()['results'][0]['committees']
    
    committee_id_list = []
    committee_name_list = []
    subcommittee_id_list = []
    subcommittee_name_list = []
    subcommitte_parent_id_list = []
    
    for committee in committees:
        committee_id_list.append(committee['id'])
        committee_name_list.append(committee['name'])
        for subcommittee in committee['subcommittees']:
            subcommittee_id_list.append(subcommittee['id'])
            subcommittee_name_list.append(subcommittee['name'])
            subcommitte_parent_id_list.append(committee['id'])
    
    committee_df = pd.DataFrame({'committee_id': committee_id_list,
                                 'committee_name': committee_name_list})
    committee_df['congress'] = congress
    committee_df['chamber'] = chamber
    subcommittee_df = pd.DataFrame({'subcommittee_id': subcommittee_id_list,
                                    'subcommittee_name': subcommittee_name_list,
                                    'parent_committee_id': subcommitte_parent_id_list})
    subcommittee_df['congress'] = congress
    subcommittee_df['chamber'] = chamber
    
    return committee_df, subcommittee_df

In [29]:
"""get committees and subcommittees for 116th congress"""

house_116_committees = get_committees_and_subcommittees(116, 'house')[0]
house_116_subcommittees = get_committees_and_subcommittees(116, 'house')[1]
senate_116_committees = get_committees_and_subcommittees(116, 'senate')[0]
senate_116_subcommittees = get_committees_and_subcommittees(116, 'senate')[1]
joint_116_committees = get_committees_and_subcommittees(116, 'joint')[0]
joint_116_subcommittees = get_committees_and_subcommittees(116, 'joint')[1]
committees_116 = pd.concat([house_116_committees, senate_116_committees, joint_116_committees])
subcommittees_116 = pd.concat([house_116_subcommittees, senate_116_subcommittees, joint_116_subcommittees])

In [31]:
"""get committees and subcommittees for 117th congress"""
house_117_committees = get_committees_and_subcommittees(117, 'house')[0]
house_117_subcommittees = get_committees_and_subcommittees(117, 'house')[1]
senate_117_committees = get_committees_and_subcommittees(117, 'senate')[0]
senate_117_subcommittees = get_committees_and_subcommittees(117, 'senate')[1]
joint_117_committees = get_committees_and_subcommittees(117, 'joint')[0]
joint_117_subcommittees = get_committees_and_subcommittees(117, 'joint')[1]
committees_117 = pd.concat([house_117_committees, senate_117_committees, joint_117_committees])
subcommittees_117 = pd.concat([house_117_subcommittees, senate_117_subcommittees, joint_117_subcommittees])


In [36]:
"""get committees and subcommittees for 118th congress"""
house_118_committees = get_committees_and_subcommittees(118, 'house')[0]
house_118_subcommittees = get_committees_and_subcommittees(118, 'house')[1]
senate_118_committees = get_committees_and_subcommittees(118, 'senate')[0]
senate_118_subcommittees = get_committees_and_subcommittees(118, 'senate')[1]
joint_118_committees = get_committees_and_subcommittees(118, 'joint')[0]
joint_118_subcommittees = get_committees_and_subcommittees(118, 'joint')[1]
committees_118 = pd.concat([house_118_committees, senate_118_committees, joint_118_committees])
subcommittees_118 = pd.concat([house_118_subcommittees, senate_118_subcommittees, joint_118_subcommittees])


In [38]:
committees_117.shape, committees_116.shape, committees_118.shape


((49, 4), (49, 4), (50, 4))

In [39]:
subcommittees_117.shape, subcommittees_116.shape, subcommittees_118.shape

((210, 5), (210, 5), (210, 5))

In [71]:
committees = pd.concat([committees_116, committees_117, committees_118])
subcommittees = pd.concat([subcommittees_116, subcommittees_117, subcommittees_118])
committees

,committee_id,committee_name,congress,chamber
0,HSHA,Committee on Administration,116,house
1,HSAG,Committee on Agriculture,116,house
2,HSAP,Committee on Appropriations,116,house
3,HSAS,Committee on Armed Services,116,house
4,HSED,Committee on Education and the Workforce,116,house
...,...,...,...,...
0,JCSE,Commission on Security and Cooperation in Europe,118,joint
1,JSPR,Joint Committee on Printing,118,joint
2,JSTX,Joint Committee on Taxation,118,joint
3,JSLC,Joint Committee on the Library,118,joint


In [72]:
"""get committee members"""
committee_assignment_df_list = []
for index, row in committees.iterrows():
    congress = row['congress']
    chamber = row['chamber']
    committee_id = row['committee_id']
    url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}.json'
    response = requests.get(url, headers=headers)
    members = response.json()['results'][0]['current_members']
    member_id_list = []
    member_name_list = []
    member_begin_date_list = []
    for member in members:
        member_id_list.append(member['id'])
        member_name_list.append(member['name'])
        member_begin_date_list.append(member['begin_date'])
    
    committee_members_df = pd.DataFrame({'member_id': member_id_list,
                                         'member_name': member_name_list,
                                         'member_begin_date': member_begin_date_list})
    committee_members_df['congress'] = congress
    committee_members_df['chamber'] = chamber
    committee_members_df['committee_id'] = committee_id
    committee_assignment_df_list.append(committee_members_df)
        
        
committee_assignments = pd.concat(committee_assignment_df_list)


In [74]:
committee_assignments

,member_id,member_name,member_begin_date,congress,chamber,committee_id
0,D000619,Rodney Davis,2019-01-09,116,house,HSHA
1,L000397,Zoe Lofgren,2019-01-09,116,house,HSHA
2,W000819,Mark Walker,2019-02-06,116,house,HSHA
3,R000606,Jamie Raskin,2019-01-29,116,house,HSHA
4,L000583,Barry Loudermilk,2019-02-06,116,house,HSHA
...,...,...,...,...,...,...
15,M000317,Nicole Malliotakis,None,118,joint,JSEC
16,B001292,Donald Beyer,None,118,joint,JSEC
17,T000483,David Trone,None,118,joint,JSEC
18,M001160,Gwen Moore,None,118,joint,JSEC


In [67]:
"""get subcommittee members"""
subcommittee_assignment_df_list = []

for index, row in subcommittees.iterrows():
    congress = row['congress']
    chamber = row['chamber']
    committee_id = row['parent_committee_id']
    subcommittee_id = row['subcommittee_id']
    url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}/subcommittees/{subcommittee_id}.json'
    response = requests.get(url, headers=headers)
    members = response.json()['results'][0]['current_members']
    member_id_list = []
    member_name_list = []
    member_begin_date_list = []
    for member in members:
        member_id_list.append(member['id'])
        member_name_list.append(member['name'])
        member_begin_date_list.append(member['begin_date'])
    
    subcommittee_members_df = pd.DataFrame({'member_id': member_id_list,
                                         'member_name': member_name_list,
                                         'member_begin_date': member_begin_date_list})
    subcommittee_members_df['congress'] = congress
    subcommittee_members_df['chamber'] = chamber
    subcommittee_members_df['committee_id'] = committee_id
    subcommittee_members_df['subcommittee_id'] = subcommittee_id
    subcommittee_assignment_df_list.append(subcommittee_members_df)

subcommittee_assignments = pd.concat(subcommittee_assignment_df_list)


,member_id,member_name,member_begin_date,congress,chamber,committee_id,subcommittee_id
0,F000455,Marcia L. Fudge,2019-02-07,116,house,HSHA,HSHA01
1,D000619,Rodney Davis,2019-02-07,116,house,HSHA,HSHA01
2,B001251,G. K. Butterfield,2019-02-07,116,house,HSHA,HSHA01
3,A000371,Pete Aguilar,2019-02-07,116,house,HSHA,HSHA01
0,F000455,Marcia L. Fudge,None,116,house,HSHA,HSHA08
...,...,...,...,...,...,...,...
1,B001243,Marsha Blackburn,None,118,senate,SSJU,SSJU27
2,K000393,John Kennedy,None,118,senate,SSJU,SSJU27
3,B001277,Richard Blumenthal,None,118,senate,SSJU,SSJU27
4,H001089,Joshua Hawley,None,118,senate,SSJU,SSJU27


In [70]:
subcommittee_assignments.to_csv('subcommittee_assignments.csv', index=False)


In [75]:
committee_assignments.to_csv('committee_assignments.csv', index=False)

In [233]:
"""adding in new member info"""

new_members = pd.read_csv('../prod_data/new_ids.csv')

len(new_members['id'].unique())

63

In [217]:
committee_assignments = pd.read_csv('committee_assignments.csv')
subcommittee_assignments = pd.read_csv('subcommittee_assignments.csv')

In [218]:

transaction_ids = new_members['id'].unique()

In [219]:
len(transaction_ids)

63

In [220]:
new_committee_assignments_of_interest = committee_assignments[committee_assignments['member_id'].isin(transaction_ids)]
new_subcommittee_assignments_of_interest = subcommittee_assignments[subcommittee_assignments['member_id'].isin(transaction_ids)]
new_committee_assignments_of_interest.shape, new_subcommittee_assignments_of_interest.shape

((730, 6), (1189, 7))

In [221]:
old_committee_assignments = pd.read_csv('../prod_data/committee_assignments_of_interest.csv')
old_subcommittee_assignments = pd.read_csv('../prod_data/subcommittee_assignments_of_interest.csv')

In [222]:
old_committee_assignments.shape, old_subcommittee_assignments.shape

((802, 6), (929, 7))

In [225]:
committee_assignments_of_interest = pd.concat([committee_assignments_of_interest, old_committee_assignments])
subcommittee_assignments_of_interest = pd.concat([subcommittee_assignments_of_interest, old_subcommittee_assignments])

In [227]:
committee_assignments_of_interest.shape, subcommittee_assignments_of_interest.shape

((1532, 6), (2118, 7))

In [228]:
subcommittee_assignments_of_interest.to_csv('../prod_data/subcommittee_assignments_of_interest.csv', index=False)
committee_assignments_of_interest.to_csv('../prod_data/committee_assignments_of_interest.csv', index=False)

In [84]:
committee_assignments_of_interest

,member_id,member_name,member_begin_date,congress,chamber,committee_id
1,L000397,Zoe Lofgren,2019-01-09,116,house,HSHA
3,R000606,Jamie Raskin,2019-01-29,116,house,HSHA
5,D000598,Susan A. Davis,2019-01-29,116,house,HSHA
0,C001062,K. Michael Conaway,2019-01-04,116,house,HSAG
4,S001189,Austin Scott,2019-01-23,116,house,HSAG
...,...,...,...,...,...,...
12,W000800,Peter Welch,None,118,joint,JSEC
13,S001199,Lloyd Smucker,None,118,joint,JSEC
15,M000317,Nicole Malliotakis,None,118,joint,JSEC
16,B001292,Donald Beyer,None,118,joint,JSEC


In [86]:
committees.to_csv('../prod_data/committees.csv', index=False)


In [88]:
subcommittees.to_csv('../prod_data/subcommittees.csv', index=False)

In [230]:
new_members_id = new_members['id'].unique()

new_members_id

array(['C000174', 'M000934', 'C001047', 'C001035', 'P000595', 'R000307',
       'T000461', 'I000024', 'S001217', 'R000608', 'W000805', 'C001075',
       'M000355', 'W000779', 'B001277', 'W000802', 'P000612', 'K000383',
       'B001236', 'B001267', 'L000594', 'C001088', 'K000384', 'S001181',
       'J000293', 'S001198', 'S001203', 'W000437', 'H001061', 'F000062',
       'A000360', 'B001135', 'B000575', 'K000393', 'C001098', 'U000039',
       'R000605', 'R000122', 'T000476', 'D000622', 'B001261', 'P000449',
       'C000141', 'M001183', 'C000880', 'M001111', 'D000618', 'V000128',
       'C001056', 'C000127', 'E000285', 'B001288', 'W000817', 'S000320',
       'T000278', 'H000273', 'H000601', 'F000463', 'G000359', 'T000250',
       'L000571', 'M001198', 'P000603'], dtype=object)

In [234]:
"""Get bills for members"""
member_bills_list = []


for member_id in new_members_id:
    type = 'introduced'
    offset_value = 0
    current_congress = 118
    member_list = []
    bill_congress_list = []
    bill_id_list = []
    bill_title_list = []
    bill_intro_date_list = []
    bill_summary_list = []
    bill_url_list = []

    url = f'https://api.propublica.org/congress/v1/members/{member_id}/bills/{type}.json?offset={offset_value}'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    results_check = response.json()
    if 'results' in response.json().keys():  
        results = response.json()['results']
        bill_number = results[0]['num_results']
        times = bill_number // 20 + 1
        while times >=0 and current_congress > 115:
            url = f'https://api.propublica.org/congress/v1/members/{member_id}/bills/{type}.json?offset={offset_value}'
            response = requests.get(url, headers=headers)
            offset_value+= 20
            times -= 1
            bills = response.json()['results'][0]['bills']
            for bill in bills:
                current_congress = int(bill['congress'])
                member_list.append(member_id)
                bill_congress_list.append(bill['congress'])
                bill_id_list.append(bill['bill_id'])
                bill_title_list.append(bill['title'])
                bill_intro_date_list.append(bill['introduced_date'])
                bill_summary_list.append(bill['summary'])
                bill_url_list.append(bill['congressdotgov_url'])
    
        bills_df = pd.DataFrame({'bill_congress': bill_congress_list,
                                 'bill_id': bill_id_list,
                                 'bill_title': bill_title_list,
                                 'bill_intro_date': bill_intro_date_list,
                                 'member_id': member_list,
                                 'bill_summary': bill_summary_list,
                                 'bill_url': bill_url_list})
    
        member_bills_list.append(bills_df)
    else:
        pass
    
member_bills = pd.concat(member_bills_list)

member_bills

,bill_congress,bill_id,bill_title,bill_intro_date,member_id,bill_summary,bill_url
0,118,s3027-118,A bill to amend the Internal Revenue Code of 1...,2023-10-04,C000174,,https://www.congress.gov/bill/118th-congress/s...
1,118,s2958-118,An original bill to amend the Coastal Barrier ...,2023-09-27,C000174,,https://www.congress.gov/bill/118th-congress/s...
2,118,s2959-118,An original bill to amend the Comprehensive En...,2023-09-27,C000174,,https://www.congress.gov/bill/118th-congress/s...
3,118,s2924-118,"A bill to amend title 31, United States Code, ...",2023-09-26,C000174,,https://www.congress.gov/bill/118th-congress/s...
4,118,sres339-118,A resolution authorizing the Sergeant at Arms ...,2023-09-12,C000174,,https://www.congress.gov/bill/118th-congress/s...
...,...,...,...,...,...,...,...
175,115,s2760-115,A bill to amend the Clean Air Act to exclude e...,2018-04-26,P000603,This bill amends the Clean Air Act to exclude ...,https://www.congress.gov/bill/115th-congress/s...
176,115,s2761-115,A bill to amend the Clean Air Act to clarify w...,2018-04-26,P000603,This bill amends the Clean Air Act to provide ...,https://www.congress.gov/bill/115th-congress/s...
177,115,sconres36-115,A concurrent resolution setting forth the cong...,2018-04-18,P000603,Establishes the congressional budget for the f...,https://www.congress.gov/bill/115th-congress/s...
178,115,sconres37-115,A concurrent resolution setting forth the cong...,2018-04-18,P000603,Establishes the congressional budget for the f...,https://www.congress.gov/bill/115th-congress/s...


In [235]:
old_bills = pd.read_csv('../prod_data/member_bills.csv')

old_bills

,bill_congress,bill_id,bill_title,bill_intro_date,member_id,bill_summary,bill_url,bill_slug
0,118,hr6345-118,To amend the Internal Revenue Code of 1986 to ...,2023-11-09,L000590,NaN,https://www.congress.gov/bill/118th-congress/h...,hr6345
1,118,hr6056-118,To direct the Secretary of Defense to submit t...,2023-10-25,L000590,NaN,https://www.congress.gov/bill/118th-congress/h...,hr6056
2,118,hr5997-118,To require the Secretary of Energy to further ...,2023-10-19,L000590,NaN,https://www.congress.gov/bill/118th-congress/h...,hr5997
3,118,hr5720-118,To assist trafficking survivors who lack gover...,2023-09-26,L000590,NaN,https://www.congress.gov/bill/118th-congress/h...,hr5720
4,118,hr5645-118,To establish a matched savings program for low...,2023-09-21,L000590,NaN,https://www.congress.gov/bill/118th-congress/h...,hr5645
...,...,...,...,...,...,...,...,...
9780,118,hr5132-118,To bolster Department of Homeland Security eff...,2023-08-01,T000488,NaN,https://www.congress.gov/bill/118th-congress/h...,hr5132
9781,118,hr4661-118,To require the Director of the Office of Entre...,2023-07-14,T000488,NaN,https://www.congress.gov/bill/118th-congress/h...,hr4661
9782,118,hr4358-118,To provide lawful permanent resident status an...,2023-06-23,T000488,NaN,https://www.congress.gov/bill/118th-congress/h...,hr4358
9783,118,hr2810-118,To designate the Federal building located at 9...,2023-04-24,T000488,NaN,https://www.congress.gov/bill/118th-congress/h...,hr2810


In [236]:
member_bills['bill_slug'] = member_bills['bill_id'].apply(lambda x: x.split('-')[0])
member_bills

,bill_congress,bill_id,bill_title,bill_intro_date,member_id,bill_summary,bill_url,bill_slug
0,118,s3027-118,A bill to amend the Internal Revenue Code of 1...,2023-10-04,C000174,,https://www.congress.gov/bill/118th-congress/s...,s3027
1,118,s2958-118,An original bill to amend the Coastal Barrier ...,2023-09-27,C000174,,https://www.congress.gov/bill/118th-congress/s...,s2958
2,118,s2959-118,An original bill to amend the Comprehensive En...,2023-09-27,C000174,,https://www.congress.gov/bill/118th-congress/s...,s2959
3,118,s2924-118,"A bill to amend title 31, United States Code, ...",2023-09-26,C000174,,https://www.congress.gov/bill/118th-congress/s...,s2924
4,118,sres339-118,A resolution authorizing the Sergeant at Arms ...,2023-09-12,C000174,,https://www.congress.gov/bill/118th-congress/s...,sres339
...,...,...,...,...,...,...,...,...
175,115,s2760-115,A bill to amend the Clean Air Act to exclude e...,2018-04-26,P000603,This bill amends the Clean Air Act to exclude ...,https://www.congress.gov/bill/115th-congress/s...,s2760
176,115,s2761-115,A bill to amend the Clean Air Act to clarify w...,2018-04-26,P000603,This bill amends the Clean Air Act to provide ...,https://www.congress.gov/bill/115th-congress/s...,s2761
177,115,sconres36-115,A concurrent resolution setting forth the cong...,2018-04-18,P000603,Establishes the congressional budget for the f...,https://www.congress.gov/bill/115th-congress/s...,sconres36
178,115,sconres37-115,A concurrent resolution setting forth the cong...,2018-04-18,P000603,Establishes the congressional budget for the f...,https://www.congress.gov/bill/115th-congress/s...,sconres37


In [238]:
totla_bills = pd.concat([member_bills, old_bills])

totla_bills.to_csv('../prod_data/member_bills.csv', index=False)

In [164]:
member_bills.to_csv('../prod_data/member_bills.csv', index=False)

In [203]:
member_bills['bill_id'].value_counts()

bill_id
hr6345-118    1
hr7660-117    1
hr8367-117    1
hr8271-117    1
hr8190-117    1
             ..
hr2795-116    1
hr1960-116    1
hr1606-116    1
hr1442-116    1
hr306-118     1
Name: count, Length: 9785, dtype: int64

In [194]:
url = f'https://api.propublica.org/congress/v1/members/L000397/statements/116.json'
response = requests.get(url, headers=headers)
response.json()

{'status': 'OK',
 'copyright': 'Copyright (c) 2023 Pro Publica Inc. All Rights Reserved.',
 'num_results': 155,
 'member_id': 'L000397',
 'congress': 116,
 'member_uri': 'https://api.propublica.org/congress/v1/members/L000397.json',
 'name': 'Zoe Lofgren',
 'offset': 0,
 'results': [{'url': 'https://lofgren.house.gov/media/press-releases/chair-lofgren-statement-fairness-high-skilled-immigrants-act',
   'date': '2020-12-21',
   'title': 'Chair Lofgren Statement on the Fairness for High-Skilled Immigrants Act',
   'statement_type': 'Press Release',
   'chamber': 'House',
   'state': 'CA',
   'party': 'D',
   'subjects': []},
  {'url': 'https://lofgren.house.gov/media/press-releases/chair-lofgren-senate-changes-fairness-high-skilled-immigrants-act',
   'date': '2020-12-03',
   'title': 'Chair Lofgren on Senate Changes to the Fairness for High-Skilled Immigrants Act',
   'statement_type': 'Press Release',
   'chamber': 'House',
   'state': 'CA',
   'party': 'D',
   'subjects': []},
  {'url

In [199]:
"""Get member statements"""
committee_assignments_of_interest


,member_id,member_name,member_begin_date,congress,chamber,committee_id
1,L000397,Zoe Lofgren,2019-01-09,116,house,HSHA
3,R000606,Jamie Raskin,2019-01-29,116,house,HSHA
5,D000598,Susan A. Davis,2019-01-29,116,house,HSHA
0,C001062,K. Michael Conaway,2019-01-04,116,house,HSAG
4,S001189,Austin Scott,2019-01-23,116,house,HSAG
...,...,...,...,...,...,...
12,W000800,Peter Welch,None,118,joint,JSEC
13,S001199,Lloyd Smucker,None,118,joint,JSEC
15,M000317,Nicole Malliotakis,None,118,joint,JSEC
16,B001292,Donald Beyer,None,118,joint,JSEC


In [201]:
"""get hearings for each committee"""
congress = 116
chamber = 'house'
committee_id = 'HSAG'
url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}/hearings.json'
response = requests.get(url, headers=headers)
hearings = response.json()['results'][0]['hearings']

hearing_congress = congress
hearing_chamber = chamber
hearing_committee_id = committee_id
hearing_date = hearings['date']
hearing_description = hearings['description']





{'status': 'OK',
 'copyright': 'Copyright (c) 2023 Pro Publica Inc. All Rights Reserved.',
 'results': [{'congress': '116',
   'num_results': 20,
   'offset': 0,
   'hearings': [{'chamber': 'House',
     'committee': 'Committee on Agriculture',
     'committee_code': 'HSAG',
     'api_uri': 'https://api.propublica.org/congress/v1/116/house/committees/HSAG.json',
     'date': '2020-12-09',
     'time': '10:00:00',
     'location': '',
     'description': '1890 Land Grant Institutions 130 Years of Building Equity in Agriculture',
     'bill_ids': [],
     'url': 'https://docs.house.gov/Committee/Calendar/ByEvent.aspx?EventID=111113',
     'meeting_type': 'Hearing'},
    {'chamber': 'House',
     'committee': 'Committee on Agriculture',
     'committee_code': 'HSAG',
     'api_uri': 'https://api.propublica.org/congress/v1/116/house/committees/HSAG.json',
     'date': '2020-10-01',
     'time': '10:00:00',
     'location': 'LHOB 1300',
     'description': 'Challenges and Successes of Conse

In [208]:
hearings_dfs = []
for index, row in committee_assignments_of_interest.head(5).iterrows():
    hearing_congress_list = []
    hearing_chamber_list = []
    hearing_committee_id_list = []
    hearing_date_list = []
    hearing_description_list = []
    
    offset_value = 0
    congress = row['congress']
    chamber = row['chamber']
    committee_id = row['committee_id']
    url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}/hearings.json'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    results_check = response.json()

    if 'results' in response.json().keys():
        results = response.json()['results']
        bill_number = results[0]['num_results']
        times = bill_number // 20 + 1
        while times >=0:
            url = f'https://api.propublica.org/congress/v1/{congress}/{chamber}/committees/{committee_id}/hearings.json?offset={offset_value}'
            response = requests.get(url, headers=headers)
            hearings = response.json()['results'][0]['hearings']
            
            for hearing in hearings:
                hearing_congress_list.append(congress)
                hearing_chamber_list.append(chamber)
                hearing_committee_id_list.append(committee_id)
                hearing_date_list.append(hearing['date'])
                hearing_description_list.append(hearing['description'])
            offset_value+= 20
            times -= 1
        hearings_df = pd.DataFrame({'hearing_congress': hearing_congress_list,
                                     'hearing_chamber': hearing_chamber_list,
                                     'hearing_committee_id': hearing_committee_id_list,
                                     'hearing_date': hearing_date_list,
                                     'hearing_description': hearing_description_list})
        hearings_dfs.append(hearings_df)
        
    else:
        pass
    
hearings_df = pd.concat(hearings_dfs)

   

In [209]:
hearings_df

,hearing_congress,hearing_chamber,hearing_committee_id,hearing_date,hearing_description
0,116,house,HSHA,2019-02-14,For the People: Our American Democracy
1,116,house,HSHA,2019-02-07,Organizational Meeting
0,116,house,HSHA,2019-02-14,For the People: Our American Democracy
1,116,house,HSHA,2019-02-07,Organizational Meeting
0,116,house,HSHA,2019-02-14,For the People: Our American Democracy
...,...,...,...,...,...
42,116,house,HSAG,2019-04-30,&quot;Reviewing the State of the Dairy Economy...
43,116,house,HSAG,2019-04-03,Examining the Proposed ABAWD Rule and its Impa...
44,116,house,HSAG,2019-02-27,To receive testimony from Secretary Sonny Perd...
45,116,house,HSAG,2019-02-27,the Budget Views and Estimates Letter of the C...
